# Video: Asking a Language Model Yes or No Questions

Language models are built around the simple task of generating one token of text at a time.
In this video, we will apply language models to answer simple yes or no questions.

The simplest way to extract information from a language model is to ask it a series of yes or no questions.
In this video, I will show you some beginning techniques for querying a language model using Google's Gemini API.

In [ ]:
%pip install -q google-genai

Script:
* First, I will install Google's genai module.
* I should warn you that this is not the only generative AI module written by Google.
* The previous one was was called google-generativeai and most language modules used to help with coding get the old and new modules mixed up as of October 2025.

In [ ]:
import google.genai as genai
from google.colab import userdata

Script:
* I'm recording this screencast using Google Colab, and using their user data API to protect my API key.
* You can access your own keys by clicking on the key icon on the left.

In [ ]:
client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))

Script:
* This setups the API client object that I will use to access the Gemini API.
* A big difference from the previous API was that the old API clients were configured with a specific model.
* This client object can switch models with each request, but you will need to specify the model with each request too.
* Which model will we use?

In [ ]:
# Rate limit documentation https://ai.google.dev/gemini-api/docs/rate-limits
model_name = 'gemini-2.0-flash-lite'

Script:
* I chose gemini-2.0-flash-lite for this example.
* It's a small and relatively fast model, with a decent amount of free API quota so you can use practice with it.
* It does have the downside that it tends to be busy more often.

In [ ]:
response = client.models.generate_content(model=model_name,
                                          contents="Is chicken puttanesca a food?")
response.text

"Yes, chicken puttanesca is a food. It's a variation on the classic Italian pasta dish, puttanesca, which traditionally features tomatoes, olives, capers, garlic, and anchovies. In the chicken version, chicken is added as a protein source.\n"

Script:
* Here's how you use the language model through the API.
* I am going to call the generate method here passing in the model name and a contents parameter with the prompt.
* The response object will have the output in the text attribute.
* Let's wrap that up in a function to make it easier to test many questions.

In [ ]:
def get_response(contents):
    response = client.models.generate_content(model=model_name,
                                              contents=contents)
    return response.text

In [ ]:
get_response("Is chicken puttanesca a food?")

"Yes, chicken puttanesca is a food. It's a variation of the classic Italian pasta dish, spaghetti alla puttanesca, but made with chicken as the protein.\n\nSo, to be clear:\n\n*   **Puttanesca** is a well-known sauce made with ingredients like tomatoes, olives, capers, anchovies, garlic, and chili flakes.\n*   **Chicken puttanesca** uses that sauce and incorporates chicken."

Script:
* Much easier now.
* Let's tweak the prompt so the model reply with a short yes or no answer.
* Most language model APIs charge based on the number of input and output tokens, and we won't gain much from extended output if we just look at the first few words.

In [ ]:
get_response("Is chicken puttanesca a food? Just say yes or no.")

'Yes\n'

Script:
* That's better.
* Let's add another layer of abstraction that parses the output to return a boolean value.
* True or false instead of text that maybe starts with yes or no.

In [ ]:
def get_boolean_response(contents):
    response = get_response(contents)
    print("RESPONSE", response)

    output = response.lower().startswith("yes")
    print("OUTPUT", output)

    return output

Script:
* I added a little bit of logging here for visibility.
* We would not normally have all that output in a production system.
* One thing to not here is that the function returns True if and only if the output starts with yes.
* So if the output does not start with yes or no, it will return False.
* That will happen if the text starts with yup, sure, or of course.
* And it will return True if the response starts with yesterday.
* I've gotten a lot of mileage with this simple parsing, but it is not always enough on its own, and you often need to specify that kind of output in the prompt for reliability.
* Let's test it out.
* First, let's ask a new question.

In [ ]:
get_response("Is the sky blue?")

'Yes, the sky appears blue. This is due to a phenomenon called Rayleigh scattering, where shorter wavelengths of light (like blue and violet) are scattered more than longer wavelengths (like red and orange) by the air molecules in the atmosphere.\n'

Script:
* With this output, the new get_boolean_response should work fine.

In [ ]:
get_boolean_response("Is the sky blue?")

RESPONSE Yes, the sky is generally perceived as blue. This is due to a phenomenon called Rayleigh scattering, where shorter wavelengths of light (like blue and violet) are scattered more by the atmosphere's molecules than longer wavelengths (like red and orange).

OUTPUT True


True

Script:
* It is indeed true that the sky is blue.
* Let's ask a funky version of this question.
* Is the sky red?

In [ ]:
get_boolean_response("Is the sky red?")

RESPONSE The sky can appear red, but it's not always red. 

Here's why:

*   **Sunrise and Sunset:** The most common time to see a red sky is during sunrise and sunset. Sunlight travels a longer distance through the atmosphere at these times. This longer path scatters the blue light (which is why the sky is blue during the day) and allows the red and orange light to pass through, reaching our eyes.
*   **Weather Conditions:** Dust particles, pollution, and clouds can also affect the color of the sky, enhancing the red hues.
*   **Time of Day:** The sky's color varies depending on the time of day, atmospheric conditions, and the observer's location.

So, while the sky isn't *always* red, it *can* be, especially during sunrise and sunset.

OUTPUT False


False

Script:
* Take a close look at this response.
* The output was false, but that was the default answer.
* The text did not have a clear yes or no, and the trivial parsing defaulted to no.

In [ ]:
get_boolean_response("Is the sky red? Just answer yes or no.")

RESPONSE No

OUTPUT False


False

Script:
* If we follow the question with "Just answer yes or no", the response is no.
* So now the whole function returned False for the right reason.
* Let's ask a deeper question.

In [ ]:
get_boolean_response("Is BU better than BC?")

RESPONSE Whether Boston University (BU) is "better" than Boston College (BC) depends entirely on what you're looking for in a university. Here's a breakdown of some key differences and considerations to help you decide which might be a better fit for *you*:

**Academics:**

*   **BU:** Offers a wider range of programs and schools, including strong programs in communications, engineering, and hospitality. Known for its research opportunities and emphasis on practical experience. Strong in pre-professional programs (like pre-med and pre-law). More flexible curriculum.
*   **BC:** Known for its strong core curriculum focused on the liberal arts. Strong programs in business, law, and theology. Emphasis on Jesuit values and a more structured academic environment.
*   **Overall:** Both are excellent universities. The "better" academic program depends on your specific field of study and learning style.

**Campus and Location:**

*   **BU:** Located in the city of Boston along Commonwealth Ave

False

Script:
* In case you are not familiar, Boston University and Boston College are long term rivals in sports.
* This long-winded answer was unsatisfying, and the answer was obviously wrong.
* Let's try again with a better prompt.

In [ ]:
get_boolean_response("Is BU better than BC? Just say yes or no.")

RESPONSE This is a subjective question, so I can't give a definitive "yes" or "no". It depends on what someone is looking for in a university.

OUTPUT False


False

Script:
* Sigh.
* Sometimes you need to give the language model more hints about the right answer.

In [ ]:
get_boolean_response("Is BU better than BC at hockey?")

RESPONSE Historically, Boston University (BU) has a slight edge in men's hockey, but it's very close. Here's a breakdown:

*   **National Championships:** BU has won 5 NCAA championships, while Boston College (BC) has won 5.
*   **Recent Success:** Both teams have had periods of dominance.
*   **Head-to-Head:** The series record between the two teams is very close, making specific season results and recent games often the deciding factor in determining which is "better" in the moment.
*   **Recruiting:** Both schools are consistently strong in recruiting.

**In Conclusion:** It's a matter of opinion and depends on the specific criteria you're using to define "better." Both BU and BC are top-tier college hockey programs with a history of excellence. In any given year, one team might be slightly ahead based on their current roster and performance.
OUTPUT False


False

Script:
* This at least is correct although it is long-winded.

In [ ]:
get_boolean_response("Is BU better than BC at hockey? Just say yes or no.")

RESPONSE Yes

OUTPUT True


True

Script:
* With just say yes or no appended, now we have confirmed the truth.
* Before wrapping up, I'm going to show you how to pass in extra information to the Gemini API.
* All the language model APIs that I've used have some version of this extra configuration.

In [ ]:
from google.genai import types

Script:
* This types module gives us access to the configuration data structures.

In [ ]:
response = client.models.generate_content(model=model_name,
                               contents="Is BU better than BC at hockey?",
                               config=types.GenerateContentConfig(max_output_tokens=1))
response.text


'Historically'

Script:
* I just added one constraint to limit the number of output tokens so that the response must be brief.
* This makes the API calls faster and cheaper.
* But you should be a little careful with this option.
* I've noticed it tend to cut off outputs mid sentence, and in this case, you can see that the model was about to rant about historical trends first.
* So it is important that your prompt encourage a short answer that fits.

In [ ]:
response = client.models.generate_content(model=model_name,
                               contents="Is BU better than BC at hockey? Just say yes or no.",
                               config=types.GenerateContentConfig(max_output_tokens=1))
response.text


'Yes'

Script:
* Adding on just say yes or no does the trick.

Script: (faculty on screen)
* It is a little bit magical that we can call an API over the Internet that gives a response from an increasingly intelligent language model.
* Since GPT-3 was first made available to the public as an API in 2020, interest and actual use of generative models have exploded.
* Welcome on board!